In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from train import LitTransformer
from pathlib import Path
from data import TransformerDataset
import torch
import yaml
from data import get_tgt_str, id2token
import pandas as pd

In [ ]:
model_path = "/home/takeru/AlphaSymbol/logs/2024-1216-1842-51/lightning_logs/version_0/checkpoints/epoch=15-step=2048.ckpt"

model = LitTransformer.load_from_checkpoint(model_path)
device = model.device
model.to(device)
model.eval()

src_vocab = model.src_vocab
tgt_vocab = model.tgt_vocab
src_max_len = model.src_max_len
tgt_max_len = model.tgt_max_len

#inputs = [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,)]
inputs = [(0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,)]
#outputs = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
outputs = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#outputs = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
#outputs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#outputs = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#inputs = [(0,), (0,)]
#outputs = [0, 0]
#outputs = [1, 1]
src_str = get_tgt_str(inputs, outputs)
src_tensor = torch.tensor([src_vocab["<sos>"]] + [src_vocab[i] for i in src_str] + [src_vocab["<eos>"]])
src_tensor = torch.cat([src_tensor, torch.tensor([src_vocab["<pad>"] for _ in range(src_max_len - src_tensor.shape[0])])])
src_tensor = src_tensor.unsqueeze(0)
src_tensor = src_tensor.to(device)

current_str = [tgt_vocab["<sos>"]]

with torch.no_grad():
    for i in range(tgt_max_len - 1):
        out_str = current_str + [tgt_vocab["<pad>"]] * (tgt_max_len - len(current_str) - 1)
        out_str = torch.tensor(out_str).to(device)
        out_str = out_str.reshape((1, -1))
        pred = model(src_tensor, out_str)
        next_c = pred[0, 0, :]
        max_values, max_indices = torch.max(next_c, axis=0)
        current_str.append(max_indices.item())
        current_str = current_str + [tgt_vocab["<pad>"]]
        if max_indices == tgt_vocab["<eos>"]:
            break

print([id2token(tgt_vocab, id) for id in out_str[0]])

# Analize each token

In [ ]:
from torch import nn
import pandas as pd
from data import TransformerDataset

df = pd.read_csv("/home/takeru/AlphaSymbol/data/prfndim/d3-a2-c3-r3-status.csv")
dataset = TransformerDataset(df)

idx = 0
src_in, src_out, tgt_out = dataset[idx][0], dataset[idx][1][:-1], dataset[idx][1][1:]
src_in, src_out, tgt_out = src_in.to(device), src_out.to(device), tgt_out.to(device)
pred_token_id = 1 #>=1

print("Input")
print(dataset.df["expr"].iloc[idx])

print("Raw Data")
print("src_in: ", src_in)
print("src_out: ", src_out)
print("tgt_out: ", tgt_out)

src_in = src_in.reshape((1, -1))
pad_tensor = torch.tensor([tgt_vocab["<pad>"] for _ in range(len(src_out) - pred_token_id)]).to(device)
src_out = torch.cat((src_out[:pred_token_id], pad_tensor)).reshape((1, -1))
tgt_out = tgt_out[pred_token_id - 1].reshape((1, ))

print("Processed data for training")
print("src_in: ", src_in)
print("src_out: ", src_out)
print("tgt_out: ", tgt_out)


model.eval()
pred = model(src_in, src_out) # (Seq, N, E)
pred_token = pred[-1, 0, :]
print("pred: ", pred_token)

    
loss_fn = nn.CrossEntropyLoss(ignore_index=tgt_vocab["<pad>"])
pred_for_loss = pred_token.reshape((1, -1))
loss = loss_fn(pred_for_loss, tgt_out)
print(loss)